# Домашнее задание № 3. Исправление опечаток

In [ ]:
!pip install textdistance razdel -q

In [26]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
import textdistance
from collections import defaultdict
from tqdm import tqdm

In [7]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [10]:
corpus = open('wiki_data.txt', encoding='utf8').read()

In [11]:
vocab = Counter(re.findall('\w+', corpus.lower()))

In [12]:
word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [13]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()

    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]

    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]

    assert len(tokens_1) == len(tokens_2)

    return list(zip(tokens_1, tokens_2))

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [20]:
def P(word, vocab):
    return vocab[word] / sum(vocab.values())

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn]

    return [(id2word[top], similarities[top]) for top in topn]

def get_closest_match_with_metric(text, lookup, vocab, topn=20, metric=textdistance.levenshtein):
    similarities = Counter()

    for word in lookup:
        similarities[word] = (metric.normalized_similarity(text, word), P(word, vocab))

    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, vocab, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, vocab, topn, metric=metric)


    return closest

In [23]:
get_closest_hybrid_match("седня", X, vec, vocab, topn=10)

[('седая', (0.8, 7.759036901785528e-07)),
 ('сегодня', (0.7142857142857143, 6.730964512298947e-05)),
 ('средняя', (0.7142857142857143, 6.226627113682887e-05)),
 ('беседня', (0.7142857142857143, 1.939759225446382e-07)),
 ('седнев', (0.6666666666666667, 1.939759225446382e-07)),
 ('бедняк', (0.6666666666666667, 1.939759225446382e-07)),
 ('соседняя', (0.625, 1.9397592254463824e-06)),
 ('середняя', (0.625, 1.939759225446382e-07)),
 ('средняго', (0.625, 1.939759225446382e-07)),
 ('дня', (0.6, 0.00014276627899285373))]

## 2.  Symspell (7 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!)
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

## Словарь удалений:

In [28]:
def get_deletes(word):
    return [word[:i] + word[i+1:] for i in range(len(word))]

In [30]:
deletes_dict = defaultdict(set)
for word in tqdm(vocab):
    for deleted in get_deletes(word):
        deletes_dict[deleted].add(word)
deletes_dict["арактеризовался"]

100%|██████████| 368802/368802 [00:13<00:00, 28074.15it/s]


{'характеризовался'}

In [35]:
deletes_dict['олько']

{'ольско', 'только'}

In [53]:
P('ольско', vocab)

1.939759225446382e-07

In [54]:
P('только', vocab)

0.0008692061089225239

## Symspell

In [ ]:
# дублирую, но пусть будет тут тоже
def P(word, vocab):
    return vocab[word] / sum(vocab.values())

In [45]:
def symspell_correct(word, deletes_dict, vocab):
    candidates = set()

    if word in deletes_dict:
        candidates.update(deletes_dict[word])

    if not candidates:
        return word

    # выбираем наиболее вероятное
    return max(candidates, key=lambda w: P(w, vocab))

In [51]:
symspell_correct('олько', deletes_dict, vocab)

'только'

## Оценка

In [55]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], symspell_correct(pair[1], deletes_dict, vocab))
            cashed[pair[1]] = pred
        else:
            pred = pair[1]


        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1

        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1

100%|██████████| 915/915 [00:03<00:00, 239.23it/s]


In [56]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8818409204602301
0.15993788819875776
0.011370161938670036
